# Food100 Classification using MobileNet V2

## Dataset : [UEC Food-100](http://foodcam.mobi/dataset100.html)

In [ ]:
import os
print(os.listdir('../input/uecfood100'))

In [ ]:
train_dir = '../input/uecfood100/UECFOOD100'

In [ ]:
dirNames = os.listdir(train_dir)
dirNames.remove('category_ja_sjis.txt')

for dirName in dirNames:
    if '.txt' in dirName:
        print(dirName)
        dirNames.remove(dirName)


print(dirNames)
print(len(dirNames))

### get labels from category.txt

In [ ]:
import pandas as pd
df = pd.read_csv(train_dir+'/category.txt', sep='\t')
print(df)

labels = df['name'].tolist()

In [ ]:
print(dirNames)

In [ ]:
trainFiles = []
trainClasses = []

for dirName in dirNames:
    for file in os.listdir(train_dir+"/"+dirName):
        trainFiles.append(train_dir+"/"+dirName+"/"+file)
        trainClasses.append(dirName)

print(len(trainFiles), len(trainClasses))

### display image of a training data

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

In [ ]:
plt.imshow(mpimg.imread(trainFiles[0]))

## Dataset Equilibre 

In [ ]:
from collections import Counter
import seaborn as sns
sns.set_style("whitegrid")

def plot_equilibre(categories, counts):

    plt.figure(figsize=(12, 8))

    sns_bar = sns.barplot(x=categories, y=counts)
    sns_bar.set_xticklabels(categories, rotation=45)
    plt.title('Equilibre of Training Dataset')
    plt.show()

In [ ]:
categories = dirNames
counts = []
[counts.append(trainClasses.count(dirName)) for dirName in dirNames]

plot_equilibre(categories, counts)

## Data Augmentation

In [ ]:
import numpy as np
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from tensorflow.keras.utils import to_categorical

from IPython.display import Image
import matplotlib.pyplot as plt

In [ ]:
target_size=(224,224)
batch_size = 16

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.15,
    zoom_range=0.15,
    horizontal_flip=True,
    vertical_flip=True)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=target_size,
    batch_size=batch_size,
    color_mode='rgb',    
    shuffle=True,
    seed=42,
    class_mode='categorical')


## Build Model

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.models import Model, save_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import Input, BatchNormalization, Activation, LeakyReLU, Concatenate
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
num_classes = 100
input_shape = (224,224,3)

In [ ]:
# Build Model
net = MobileNetV2(input_shape=input_shape, weights='imagenet', include_top=False)

# add two FC layers (with L2 regularization)
x = net.output
x = GlobalAveragePooling2D()(x) 
#x = BatchNormalization()(x)

#x = Dense(256, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(x)
x = Dense(1024)(x)
#x = Dropout(0.2)(x)

#x = Dense(256, kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01))(x)
x = Dense(256)(x)
#x = Dropout(0.2)(x)

# Output layer
out = Dense(num_classes, activation="softmax")(x)

model = Model(inputs=net.input, outputs=out)
model.summary()

## Transfer Learning

In [ ]:
# Compile Model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
## set Checkpoint : save best only, verbose on
#checkpoint = ModelCheckpoint("food100_mobilenetv2.hdf5", monitor='accuracy', verbose=0, save_best_only=True, mode='auto', save_freq=1)

## Train Model

In [ ]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
num_epochs = 20

In [ ]:
# Train Model
history = model.fit_generator(train_generator,steps_per_epoch=STEP_SIZE_TRAIN,epochs=num_epochs) #, callbacks=[checkpoint])

## Save Model

In [ ]:
## Save Model
save_model(model, 'food100_mobilenetv2.h5')

In [ ]:
## load best model weights if using callback (save-best-only)
#model.load_weights("food100_mobilenetv2.hdf5")

## Evaluate Model

In [ ]:
#score = model.evaluate_generator(test_generator, steps=STEP_SIZE_TEST)
#print(score)

## Confusion Matrix

In [ ]:
#predY=model.predict_generator(test_generator)
#y_pred = np.argmax(predY,axis=1)
##y_label= [labels[k] for k in y_pred]
#y_actual = test_generator.classes
#cm = confusion_matrix(y_actual, y_pred)
#print(cm)

### report confusion matrix

In [ ]:
#print(classification_report(y_actual, y_pred, target_names=labels))

## Test Model

### try one image

In [ ]:
dir_name  = '1'
file_name = '1.jpg'
testfile  = train_dir+'/'+dir_name+'/'+file_name
plt.imshow(mpimg.imread(testfile))

In [ ]:
from PIL import Image
def prepare_image(filepath):
    img = Image.open(filepath)  
    out = img.resize((224, 224)) # (width, height), resample
    return out

In [ ]:
img = prepare_image(testfile)
testData = np.array(img).reshape(1,224,224,3)
testData = testData / 255.0
predictions = model.predict(testData)
print(predictions[0])

In [ ]:
maxindex = int(np.argmax(predictions[0]))
print('Predicted: %s, Probability = %f' %(labels[maxindex], predictions[0][maxindex]) )